In [15]:
import pandas as pd
import numpy as np

In [3]:

# 读取 CSV 文件
df = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\CSIRO\OOF\oof_bad_cases_top357.csv")

# 查看前几行数据
print(df.head())

print(df.columns.to_list)

             ID              image_path Sampling_Date State   Species  \
0  ID1357758282  train/ID1357758282.jpg     2015/1/15   NSW   Lucerne   
1   ID384648061   train/ID384648061.jpg     2015/10/6   NSW    Fescue   
2  ID1540480250  train/ID1540480250.jpg     2015/2/24   NSW  Phalaris   
3  ID1139866256  train/ID1139866256.jpg      2015/5/7   NSW    Fescue   
4  ID1121692672  train/ID1121692672.jpg     2015/2/24   NSW  Phalaris   

   Pre_GSHH_NDVI  Height_Ave_cm  Dry_Clover_g  Dry_Dead_g  Dry_Green_g  ...  \
0           0.77        62.0000           0.0      4.8000     107.3000  ...   
1           0.85        14.6667           0.0     27.7164     157.9836  ...   
2           0.58        20.0000           0.0      3.6485     116.7515  ...   
3           0.66        13.0000           0.0     83.8407      74.0593  ...   
4           0.52        38.0000           0.0      1.3958     109.8042  ...   

   pred_Dry_Dead_g  err_Dry_Dead_g  abs_err_Dry_Dead_g  pred_GDM_g  err_GDM_g  \
0    

In [8]:
# 列名列表
cols = df.columns.tolist()

# 指定的五个目标列名（需要按照原始值、预测值和绝对误差排序）
target_cols = ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']

# 创建排序列表
sorted_columns = []
sorted_columns.append('weighted_se') 
# 将目标列按照原始值、预测值和绝对误差的顺序添加到 sorted_columns 列表中
for col in target_cols:
    sorted_columns.append(col)  # 原始值
    sorted_columns.append(f'pred_{col}')  # 预测值
    sorted_columns.append(f'abs_err_{col}')  # 绝对误差值

# 把其他不属于目标列的列名添加到末尾
other_columns = [col for col in cols if col not in sorted_columns]
sorted_columns.extend(other_columns)

# 重新排序 DataFrame
df_sorted = df[sorted_columns]

# 查看排序后的 DataFrame
print(df_sorted.head())

print(df_sorted.columns.to_list())

# 保存排序后的 DataFrame
df_sorted.to_csv(r"C:\Users\Admin\Documents\GitHub\CSIRO\OOF\sorted_oof_bad_cases_top357.csv", index=False)

print("排序后的数据已保存到：C:\\Users\\Admin\\Documents\\GitHub\\CSIRO\\OOF\\sorted_oof_bad_cases_top357.csv")



   weighted_se  Dry_Green_g  pred_Dry_Green_g  abs_err_Dry_Green_g  \
0    4722.6895     107.3000          28.93750             78.36250   
1    3502.3442     157.9836         104.18750             53.79610   
2    2467.5718     116.7515          52.34375             64.40775   
3    2142.8300      74.0593          51.21875             22.84055   
4    1717.6794     109.8042          58.59375             51.21045   

   Dry_Clover_g  pred_Dry_Clover_g  abs_err_Dry_Clover_g  Dry_Dead_g  \
0           0.0           2.843206              2.843206      4.8000   
1           0.0           0.012269              0.012269     27.7164   
2           0.0           0.371313              0.371313      3.6485   
3           0.0           2.128649              2.128649     83.8407   
4           0.0           0.143195              0.143195      1.3958   

   pred_Dry_Dead_g  abs_err_Dry_Dead_g  ...  State   Species  Pre_GSHH_NDVI  \
0         0.691414            4.108586  ...    NSW   Lucerne       

In [28]:
# 进行验证

df['GDM_g_check'] = df['Dry_Green_g'] + df['Dry_Clover_g']
df['Dry_Total_g_check'] = df['Dry_Green_g'] + df['Dry_Clover_g'] + df['Dry_Dead_g']



df["err_total"] = (df["Dry_Total_g_check"] - df["Dry_Total_g"]).abs()
df["err_gdm"] = (df["GDM_g_check"] - df["GDM_g"]).abs()



# 定义误差区间和标签
bins = [0, 0.001, 0.01, 0.1, 1, 10, np.inf]
labels = ["<0.001", "0.001~0.01", "0.01~0.1", "0.1~1", "1~10", ">10"]


df["err_total_group"] = pd.cut(df["err_total"], bins=bins, labels=labels, right=False)
df["err_gdm_group"] = pd.cut(df["err_gdm"], bins=bins, labels=labels, right=False)


# 输出每个分组的样本数量，并按照 bins 顺序显示
print("\nTotal：")
print(df["err_total_group"].value_counts().sort_index())

print("\nGDM：")
print(df["err_gdm_group"].value_counts().sort_index())





Total：
err_total_group
<0.001        356
0.001~0.01      0
0.01~0.1        0
0.1~1           1
1~10            0
>10             0
Name: count, dtype: int64

GDM：
err_gdm_group
<0.001        357
0.001~0.01      0
0.01~0.1        0
0.1~1           0
1~10            0
>10             0
Name: count, dtype: int64


In [27]:

df['GDM_g_check'] = df['pred_Dry_Green_g'] + df['pred_Dry_Clover_g']
df['Dry_Total_g_check'] = df['pred_Dry_Green_g'] + df['pred_Dry_Clover_g'] + df['pred_Dry_Dead_g']



df["err_total"] = (df["Dry_Total_g_check"] - df["pred_Dry_Total_g"]).abs()
df["err_gdm"] = (df["GDM_g_check"] - df["pred_GDM_g"]).abs()



# 定义误差区间和标签
bins = [0, 0.001, 0.01, 0.1, 1, 10, np.inf]
labels = ["<0.001", "0.001~0.01", "0.01~0.1", "0.1~1", "1~10", ">10"]


df["err_total_group"] = pd.cut(df["err_total"], bins=bins, labels=labels, right=False)
df["err_gdm_group"] = pd.cut(df["err_gdm"], bins=bins, labels=labels, right=False)


# 输出每个分组的样本数量，并按照 bins 顺序显示
print("\nTotal：")
print(df["err_total_group"].value_counts().sort_index())

print("\nGDM：")
print(df["err_gdm_group"].value_counts().sort_index())


Total：
err_total_group
<0.001          0
0.001~0.01      5
0.01~0.1       17
0.1~1         159
1~10          176
>10             0
Name: count, dtype: int64

GDM：
err_gdm_group
<0.001          1
0.001~0.01      0
0.01~0.1       18
0.1~1         204
1~10          134
>10             0
Name: count, dtype: int64


In [30]:

df['GDM_g_check'] = df['pred_Dry_Green_g'] + df['pred_Dry_Clover_g']
df['Dry_Total_g_check'] = df['pred_Dry_Green_g'] + df['pred_Dry_Clover_g'] + df['pred_Dry_Dead_g']



df["err_G"] = (df["Dry_Green_g"] - df["pred_Dry_Green_g"]).abs()/df["Dry_Green_g"]
df["err_C"] = (df["Dry_Clover_g"] - df["pred_Dry_Clover_g"]).abs()/df["Dry_Clover_g"]
df["err_D"] = (df["Dry_Dead_g"] - df["pred_Dry_Dead_g"]).abs()/df["Dry_Dead_g"]



# 定义误差区间和标签
bins = [0, 0.001, 0.01, 0.1, 1, 10, np.inf]
labels = ["<0.001", "0.001~0.01", "0.01~0.1", "0.1~1", "1~10", ">10"]


df["err_G_group"] = pd.cut(df["err_G"], bins=bins, labels=labels, right=False)
df["err_C_group"] = pd.cut(df["err_C"], bins=bins, labels=labels, right=False)
df["err_D_group"] = pd.cut(df["err_D"], bins=bins, labels=labels, right=False)


# 输出每个分组的样本数量，并按照 bins 顺序显示
print("\nG：")
print(df["err_G_group"].value_counts().sort_index())

print("\nC：")
print(df["err_C_group"].value_counts().sort_index())

print("\nD：")
print(df["err_D_group"].value_counts().sort_index())


G：
err_G_group
<0.001          1
0.001~0.01      6
0.01~0.1       63
0.1~1         228
1~10           36
>10             5
Name: count, dtype: int64

C：
err_C_group
<0.001          0
0.001~0.01      3
0.01~0.1       23
0.1~1         152
1~10           43
>10             1
Name: count, dtype: int64

D：
err_D_group
<0.001          0
0.001~0.01      2
0.01~0.1       42
0.1~1         204
1~10           66
>10             3
Name: count, dtype: int64


In [38]:

def compute_cv_score(all_preds, all_targets):
    preds = np.concatenate(all_preds, axis=0)
    targets = np.concatenate(all_targets, axis=0)

    # 确保 targets 和 preds 是二维数组，如果是 1D，就 reshape 成 [B, 5]
    if targets.ndim == 1:
        targets = targets.reshape(-1, 5)
    if preds.ndim == 1:
        preds = preds.reshape(-1, 5)

    
    # print("\n【变形后（统一为二维张量）】")
    # print(f"预测值 preds:\n{preds}\n形状: {preds.shape}")
    # print(f"真实值 targets:\n{targets}\n形状: {targets.shape}")

    # 五个目标列名与对应权重（与官方相同）
    target_cols = ["Dry_Green_g", "Dry_Clover_g", "Dry_Dead_g", "GDM_g", "Dry_Total_g"]
    weights = np.array([0.1, 0.1, 0.1, 0.2, 0.5])

    # 拼接所有目标
    y_true_flat = np.concatenate([targets[i, :] for i in range(targets.shape[0])])
    y_pred_flat = np.concatenate([preds[i, :] for i in range(preds.shape[0])])

    w_flat = np.concatenate([np.full_like(targets[:, i], weights[i]) for i in range(5)])

    # print("\n【展平后的张量】")
    # print(f"展平后的真实值 y_true_flat:\n{y_true_flat}\n形状: {y_true_flat.shape}")
    # print(f"展平后的预测值 y_pred_flat:\n{y_pred_flat}\n形状: {y_pred_flat.shape}")
    # print("\n【展开后的权重张量】")
    # print(f"w_flat:\n{w_flat}\n形状: {w_flat.shape}")

    # 全局加权均值
    y_mean = np.sum(w_flat * y_true_flat) / np.sum(w_flat)
    # print("\n【全局加权均值】")
    # print(f"y_mean: {y_mean}")

    # 计算加权残差平方和与总平方和
    ss_res = np.sum(w_flat * (y_true_flat - y_pred_flat) ** 2)
    ss_tot = np.sum(w_flat * (y_true_flat - y_mean) ** 2)

    # print("\n【加权残差平方和与总平方和】")
    # print(f"ss_res（加权残差平方和）: {ss_res}")
    # print(f"ss_tot（加权总平方和）: {ss_tot}")

    # Kaggle 官方全局加权 R²
    r2_global = ss_res / ss_tot
    return r2_global

### 原始

In [39]:
target_cols = ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
pred_cols = ['pred_' + col for col in target_cols]

# 获取目标列和预测列的真实值与预测值
all_targets = [df[target_cols].values]
all_preds = [df[pred_cols].values]

# 调用 compute_cv_score 函数计算当前的 R²
current_r2 = compute_cv_score(all_preds, all_targets)
print(f"当前的全局加权 R²: {current_r2}")

当前的全局加权 R²: 0.13075849811361956


In [60]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# 假设 df 已经包含了 target_cols 和 pred_{target_cols}

# 获取目标列和预测列的真实值与预测值
target_cols = ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
pred_cols = ['pred_' + col for col in target_cols]

all_targets = df[target_cols].values
all_preds = df[pred_cols].values

# 调用 compute_cv_score 函数计算当前的 R²
current_r2 = compute_cv_score([all_preds], [all_targets])
print(f"当前的全局加权 R²: {current_r2}")

# 训练决策树回归模型来优化预测值
# 训练一个决策树回归模型（单一树模型）
tree_model = DecisionTreeRegressor(max_depth=5)  # max_depth 可以根据需要调整

# 将所有预测列作为输入特征，目标列作为输出
tree_model.fit(all_preds, all_targets)

# 使用训练好的决策树对预测结果进行后处理
all_preds_processed = tree_model.predict(all_preds)

# 计算处理后的 R²
# 注意：这里 all_preds_processed 需要是一个正确形状的二维数组
all_preds_processed = np.array(all_preds_processed)

# 计算新的 R²
processed_r2 = compute_cv_score([all_preds_processed], [all_targets])
print(f"处理后的全局加权 R²: {processed_r2}")


当前的全局加权 R²: 0.13075849811361956
处理后的全局加权 R²: 0.11768769349413852


In [62]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# 假设 df 已经包含了 target_cols 和 pred_{target_cols}

# 获取目标列和预测列的真实值与预测值
target_cols = ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
pred_cols = ['pred_' + col for col in target_cols]

all_targets = df[target_cols].values
all_preds = df[pred_cols].values

# 调用 compute_cv_score 函数计算当前的 R²
current_r2 = compute_cv_score([all_preds], [all_targets])
print(f"当前的全局加权 R²: {current_r2}")

# 训练决策树回归模型来优化预测值
# 训练一个决策树回归模型
tree_model = DecisionTreeRegressor(max_depth=10, random_state=42)

# 使用 all_preds 作为特征，all_targets 作为目标进行训练
tree_model.fit(all_preds, all_targets)

# 使用训练好的决策树对预测结果进行后处理
# 通过树的预测，我们可以得到调整后的预测值
all_preds_processed = tree_model.predict(all_preds)

# 计算处理后的 R²
# 需要确保 all_preds_processed 维度正确
all_preds_processed = np.array(all_preds_processed)

# 计算新的 R²
processed_r2 = compute_cv_score([all_preds_processed], [all_targets])
print(f"处理后的全局加权 R²: {processed_r2}")


当前的全局加权 R²: 0.13075849811361956
处理后的全局加权 R²: 0.01765794319339965


In [63]:
from sklearn.tree import export_text
# 使用 export_text 提取这棵树的规则
tree_rules = export_text(tree_model, feature_names=pred_cols)

# 输出树的规则
print(tree_rules)

|--- pred_Dry_Green_g <= 43.39
|   |--- pred_Dry_Total_g <= 33.78
|   |   |--- pred_Dry_Green_g <= 8.67
|   |   |   |--- pred_GDM_g <= 13.60
|   |   |   |   |--- pred_Dry_Dead_g <= 9.47
|   |   |   |   |   |--- pred_Dry_Total_g <= 14.70
|   |   |   |   |   |   |--- pred_Dry_Clover_g <= 1.77
|   |   |   |   |   |   |   |--- pred_Dry_Total_g <= 9.79
|   |   |   |   |   |   |   |   |--- value: [2.40, 0.00, 1.90, 2.40, 4.30]
|   |   |   |   |   |   |   |--- pred_Dry_Total_g >  9.79
|   |   |   |   |   |   |   |   |--- pred_Dry_Dead_g <= 3.36
|   |   |   |   |   |   |   |   |   |--- value: [11.90, 0.00, 1.30, 11.90, 13.20]
|   |   |   |   |   |   |   |   |--- pred_Dry_Dead_g >  3.36
|   |   |   |   |   |   |   |   |   |--- pred_Dry_Green_g <= 7.87
|   |   |   |   |   |   |   |   |   |   |--- value: [6.32, 0.00, 3.84, 6.32, 10.16]
|   |   |   |   |   |   |   |   |   |--- pred_Dry_Green_g >  7.87
|   |   |   |   |   |   |   |   |   |   |--- value: [6.60, 0.50, 0.40, 7.10, 7.50]
|   |   |   | 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# 假设 df 已经包含了 target_cols 和 pred_{target_cols}

# 目标列和预测列
target_cols = ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
pred_cols = ['pred_' + col for col in target_cols]

# 为每个目标列拟合一个简单的线性回归模型
compensation_factors = {}

for target, pred in zip(target_cols, pred_cols):
    # 提取真实值和预测值
    y_true = df[target].values
    y_pred = df[pred].values

    # 使用线性回归拟合真实值与预测值之间的关系
    model = LinearRegression()
    model.fit(y_pred.reshape(-1, 1), y_true)  # 使用预测值拟合真实值

    # 获取回归系数和截距
    compensation_factors[target] = (model.coef_[0], model.intercept_)
    print(f"补偿公式 ({target}): y = {model.coef_[0]:.4f} * pred_{target} + {model.intercept_:.4f}")

    # 根据拟合的补偿公式调整预测值
    df[f"compensated_{pred}"] = model.coef_[0] * df[pred] + model.intercept_

